# core

In [ ]:
import src.StreamPort.core as core

anas = core.Analyses()
pm = core.ProcessingMethod()
mt = core.Metadata(description="test", version="1.0")
wf = core.Workflow()

print(anas)
print(pm)
print(mt)
print(wf)

core = core.Engine()
core.print()

# Analyes

In [ ]:
import numpy as np
from src.StreamPort.core import Engine
from src.StreamPort.core import Analyses

print("1: Creates an Analyses object and prints it")

ana1 = Analyses(data_type="unknown", formats=[], data = [
  {"name": "Analysis 1", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 2", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 3", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 4", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}
])

x = Engine(analyses=ana1)

x.print()


# ProcessingMethod

In [ ]:
from src.StreamPort.core import Engine
from src.StreamPort.core import ProcessingMethod

x = Engine()

print("1: Creates a ProcessingMethod object and prints it")
mt = ProcessingMethod()
print(mt)

print("2: Adds the settings to the Engine")
x.workflow.append(mt)
x.workflow.append(mt)
print("Number of methods: ", len(x.workflow))
print("\n")

print("3: Removes the settings from the Engine")
x.workflow.pop(1)
print("Number of methods: ", len(x.workflow))
print("\n")


# ProcessingMethod dispatchment

In [ ]:
from src.StreamPort.core import Engine
from src.StreamPort.core import ProcessingMethod
from src.StreamPort.core import Analyses
import numpy as np


class UnkownNormalizeMinMax(ProcessingMethod):
    def __init__(self):
        super().__init__()
        self.data_type = "unknown"
        self.method = "Normalize"
        self.algorithm = "MinMax"
        self.input_instance = dict
        self.output_instance = dict
        self.number_permitted = float("inf")

    def run(self, analyses):
        res = analyses.data
        if len(res) > 0:
            for i in range(len(res)):
                ana = res[i]
                if not isinstance(ana, dict) or "y" not in ana:
                    print(
                        f"Skipping {ana['name']} because its data is not a dictionary with a 'y' key."
                    )
                    continue
                ana["y"] = (ana["y"] - ana["y"].min()) / (ana["y"].max() - ana["y"].min())
                res[i] = ana
            analyses.data = res
        else:
            print("No data to process.")
        return analyses


# Algorithm specific class
class UnkownNormalizeSNV(ProcessingMethod):
    def __init__(self, liftToZero=True):
        super().__init__()
        self.data_type = "unknown"
        self.method = "Normalize"
        self.algorithm = "standard_variance_normalization"
        self.input_instance = dict
        self.output_instance = dict
        self.number_permitted = float("inf")
        self.parameters = {"liftToZero": liftToZero}

    def run(self, analyses):
        res = analyses.data
        if len(res) > 0:
            for i in range(len(res)):
                ana = res[i]
                if not isinstance(ana, dict) or "y" not in ana:
                    print(
                        f"Skipping {ana['name']} because its data is not a dictionary with a 'y' key."
                    )
                    continue
                y = np.array(ana["y"])
                norm_data = (y - y.mean()) / y.std()
                if self.parameters["liftToZero"]:
                    norm_data += abs(norm_data.min())
                ana["y"] = norm_data
                res[i] = ana
            analyses.data = res
        else:
            print("No data to process.")
        return analyses

anas = Analyses(data_type="unknown", formats=[], data = [
  {"name": "Analysis 1", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 2", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 3", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)},
  {"name": "Analysis 4", "x": np.arange(1, 11), "y": np.random.randint(0, 100, 10)}
])

x = Engine(analyses=anas)

print("1: Prints the analyses results in the Engine")
for res in x.analyses.data:
    print(f"Analyses: {res["name"]}")
    print(f"{"x"}: {res["x"]}")
    print(f"{"y"}: {res["y"]}")
    print("\n")

# Create an instance of the UnknownNormalizeMinMax class
norm_method = UnkownNormalizeMinMax()

# Create an instance of the UnknownNormalizeSNV class
norm_method2 = UnkownNormalizeSNV()

# Run the settings MinMax using the analyses as argument
analyses_minmax = norm_method.run(x.analyses)

print("2: Results from settings MinMax")
for key in analyses_minmax.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

# Run the settings SNV using the engine as argument
analyses_svn = norm_method2.run(x.analyses)

print("3: Results from settings snv")
for key in analyses_svn.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

# Adds method to the Engine
x.workflow.append(norm_method)

x.print()

x.run()

print("4: Results from the engine")
for key in x.analyses.data:
    print(f"Analyses: {key['name']}")
    print(f"x: {key['x']}")
    print(f"y: {key['y']}")
    print("\n")

# Class Inheritance

In [ ]:
class Person:
  """Class docstrings go here."""

  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    """Class method docstrings go here."""
    print(self.firstname, self.lastname)

  def welcome_from_parent(self):
    print("Welcome from parent class")

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")

x.printname()

class Student(Person):
  def __init__(self, fname, lname, year):
    super().__init__(fname, lname)
    self.graduationyear = year

  def welcome(self):
    print("Welcome", self.firstname, self.lastname, "to the class of", self.graduationyear)

  def printname(self):
    print(self.firstname, self.lastname, "overwritten!")

  def printname_from_parent(self):
    super().printname()

x = Student("Mike", "Olsen", 2019)

x.printname()

x.printname_from_parent()

x.welcome()

x.welcome_from_parent()

#help(Person)

In [ ]:
import os
import numpy as np
import plotly.graph_objects as go
from src.StreamPort.core import Engine
from src.StreamPort.device.analyses import PressureCurves
from src.StreamPort.device.methods import PressureCurvesMethodAssignBatchPositionNative
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative
from src.StreamPort.device.methods import PressureCurvesMethodScaleFeaturesNative

path = "D:/ExtractedSignals"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]
#print(len(batches))

files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    files.extend(batch_files)

eng = Engine(analyses = PressureCurves(files=files[1:30]))

print(eng)

print("Number of analyses: ", len(eng.analyses.data))

#for key, item in eng.analyses.data[3].items():
#    print(f"{key}: {item}")

eng.workflow.append(PressureCurvesMethodAssignBatchPositionNative())
eng.workflow.append(PressureCurvesMethodExtractFeaturesNative())
eng.workflow.append(PressureCurvesMethodScaleFeaturesNative())

eng.run()

#eng.analyses.plot_batches()
#eng.analyses.plot_methods()

# for key, item in eng.analyses.data[3]["features"].items():
#     out_item = item
#     if isinstance(out_item, (list, tuple, np.ndarray)) and len(out_item) > 8:
#         out_item = "array of length " + str(len(out_item))
#     print(f"{key}: {out_item}")

#eng.analyses.plot_pressure_curves(indices = [3,4])
eng.analyses.plot_features()



In [4]:
eng.analyses.plot_pressure_curves()